In [3]:
import pandas as pd 
import numpy as np 
import math
from kmodes.kmodes import KModes

In [4]:
%config IPCompleter.greedy=True 

In [5]:
data = pd.read_csv("dataset.csv", error_bad_lines = False)

In [6]:
data.head()

,Unnamed: 0,Unnamed: 0.1,ID,Date,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,FBI Code,Year,Latitude,Longitude
0,0,0,4786321,2004-01-01 00:01:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,06,2004.0,41.734106,-87.563621
1,1,1,4676906,2003-03-01 00:00:00,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,9.0,26,2003.0,41.817229,-87.637328
2,2,2,4789749,2004-06-20 11:00:00,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,20,2004.0,41.915620,-87.694019
3,3,3,4789765,2004-12-30 20:00:00,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,06,2004.0,41.919054,-87.752178
4,4,4,4677901,2003-05-01 01:00:00,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,22.0,06,2003.0,41.691785,-87.635116


In [7]:
data["Year"].value_counts()

2008.0    852053
2006.0    794684
2009.0    783900
2010.0    700691
2007.0    621848
2001.0    568517
2002.0    490879
2003.0    475913
2005.0    455811
2004.0    388205
2011.0    352066
2012.0    335670
2013.0    306703
2014.0    274527
2016.0    265462
2015.0    262995
2017.0     11357
Name: Year, dtype: int64

In [8]:
data['Date'] = pd.to_datetime(data['Date'])

In [9]:
data_dropped = data.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Latitude', 'Longitude', 'ID', 'FBI Code'], axis=1)

In [10]:
data_addMonth = data_dropped.assign(Month = data_dropped["Date"].dt.month)

In [11]:
data_addDay = data_addMonth.assign(Day = data_addMonth["Date"].dt.day)

In [12]:
data_addHour = data_addDay.assign(Hour = data_addDay["Date"].dt.hour)

In [13]:
data_dropDate = data_addHour.drop(['Date'], axis=1)

In [14]:
finalDf = data_dropDate

In [15]:
finalDf.head()

,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,District,Year,Month,Day,Hour
0,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,4.0,2004.0,1,1,0
1,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,False,True,9.0,2003.0,3,1,0
2,1752,OFFENSE INVOLVING CHILDREN,AGG CRIM SEX ABUSE FAM MEMBER,RESIDENCE,False,False,14.0,2004.0,6,20,11
3,0840,THEFT,FINANCIAL ID THEFT: OVER $300,OTHER,False,False,25.0,2004.0,12,30,20
4,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,22.0,2003.0,5,1,1


In [16]:
myData = finalDf.to_csv('myData.csv')

In [38]:
km_5 = KModes(n_clusters=5, init = "Huang", n_init = 5, verbose=1)

In [39]:
km_10 = KModes(n_clusters=10, init = "Huang", n_init = 5, verbose=1)

In [40]:
km_25 = KModes(n_clusters=25, init = "Huang", n_init = 5, verbose=1)

In [17]:
def filterYear(year):
    df = finalDf[finalDf.Year==year]
    return df

In [148]:
def filterDistrict(district):
    df = finalDf[finalDf.District == district]
    return df

In [47]:
def makeClusters(year, num):
    df = filterYear(year)
    df = df.reset_index()
    if(num == 5):
        fitClusters = km_5.fit_predict(df)
        clusterCentroidsDf = pd.DataFrame(km_5.cluster_centroids_)
    elif(num == 10):
        fitClusters = km_10.fit_predict(df)
        clusterCentroidsDf = pd.DataFrame(km_10.cluster_centroids_)
    else:
        fitClusters = km_25.fit_predict(df)
        clusterCentroidsDf = pd.DataFrame(km_25.cluster_centroids_)
    clusterCentroidsDf.columns = df.columns
    return fitClusters, clusterCentroidsDf

In [28]:
def labelsDf(year, num, clustersDf):
    df = filterYear(year)
    df = df.reset_index()
    clustersDf = pd.DataFrame(clustersDf)
    if(num == 5):
        clustersDf.columns = ['clusters_5']
    elif(num == 10):
        clustersDf.columns = ['clusters_10']
    else:
        clustersDf.columns = ['clusters_25']
    combinedDf = pd.concat([df, clustersDf], axis = 1)
    return combinedDf

In [48]:
clusters2011_5 = makeClusters(2011, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 62300, cost: 2573770.0
Run 1, iteration: 2/100, moves: 10878, cost: 2573770.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 42006, cost: 2626185.0
Run 2, iteration: 2/100, moves: 30778, cost: 2626184.0
Run 2, iteration: 3/100, moves: 0, cost: 2626184.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 61118, cost: 2681431.0
Run 3, iteration: 2/100, moves: 0, cost: 2681431.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 83037, cost: 2610673.0
Run 4, iteration: 2/100, moves: 9409, cost: 2610673.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 63472, cost: 2534947.0
Run 5, iteration: 2/100, moves: 6320, cost: 2534947.0
Best 

In [53]:
labelsDf(2011, 5, clusters2011_5[0]).to_csv("year2011_5clusters.csv")

In [54]:
clusters2011_5[1].to_csv("year2011_5clusterCentroids.csv")

In [55]:
clusters2011_10 = makeClusters(2011, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 81089, cost: 2357670.0
Run 1, iteration: 2/100, moves: 8731, cost: 2357670.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 108087, cost: 2326715.0
Run 2, iteration: 2/100, moves: 32148, cost: 2324804.0
Run 2, iteration: 3/100, moves: 920, cost: 2324804.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 83884, cost: 2417159.0
Run 3, iteration: 2/100, moves: 8077, cost: 2417159.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 63805, cost: 2467505.0
Run 4, iteration: 2/100, moves: 7035, cost: 2467505.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 77171, cost: 2405964.0
Run 5, iteration: 2/100, moves: 33948, cost: 2371001.0

In [64]:
labelsDf(2011, 10, clusters2011_10[0]).to_csv("year2011_10clusters.csv")

In [62]:
clusters2011_10[1].to_csv("year2011_10clusterCentroids.csv")

In [68]:
clusters2011_25 = makeClusters(2011, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 49286, cost: 2219951.0
Run 1, iteration: 2/100, moves: 649, cost: 2219950.0
Run 1, iteration: 3/100, moves: 0, cost: 2219950.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 81571, cost: 2137781.0
Run 2, iteration: 2/100, moves: 38458, cost: 2134501.0
Run 2, iteration: 3/100, moves: 572, cost: 2134501.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 69690, cost: 2157806.0
Run 3, iteration: 2/100, moves: 8677, cost: 2157806.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 61085, cost: 2226666.0
Run 4, iteration: 2/100, moves: 3667, cost: 2226666.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 71742, cost: 2202535.0
Run 5

In [69]:
labelsDf(2011, 25, clusters2011_25[0]).to_csv("year2011_25clusters.csv")

In [70]:
clusters2011_25[1].to_csv("year2011_25clusterCentroids.csv")

In [71]:
clusters2012_5 = makeClusters(2012, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 80963, cost: 2499763.0
Run 1, iteration: 2/100, moves: 14076, cost: 2499763.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 102021, cost: 2413103.0
Run 2, iteration: 2/100, moves: 4636, cost: 2413103.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 106190, cost: 2398784.0
Run 3, iteration: 2/100, moves: 41096, cost: 2398783.0
Run 3, iteration: 3/100, moves: 0, cost: 2398783.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 51430, cost: 2441651.0
Run 4, iteration: 2/100, moves: 10225, cost: 2441651.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 85730, cost: 2446540.0
Run 5, iteration: 2/100, moves: 75, cost: 2446540.0
B

In [75]:
labelsDf(2012, 5, clusters2012_5[0]).to_csv("year2012_5clusters.csv")

In [78]:
clusters2012_5[1].to_csv("year2012_5clusterCentroids.csv")

In [72]:
clusters2012_10 = makeClusters(2012, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 82804, cost: 2230139.0
Run 1, iteration: 2/100, moves: 45306, cost: 2228191.0
Run 1, iteration: 3/100, moves: 2642, cost: 2228191.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 91943, cost: 2251645.0
Run 2, iteration: 2/100, moves: 23351, cost: 2248290.0
Run 2, iteration: 3/100, moves: 1290, cost: 2248290.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 77268, cost: 2258198.0
Run 3, iteration: 2/100, moves: 17733, cost: 2258198.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 52766, cost: 2248270.0
Run 4, iteration: 2/100, moves: 2375, cost: 2248270.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 64983, cost: 2200924.

In [80]:
labelsDf(2012, 10, clusters2012_10[0]).to_csv("year2012_10clusters.csv")

In [83]:
clusters2012_10[1].to_csv("year2012_10clusterCentroids.csv")

In [73]:
clusters2012_25 = makeClusters(2012, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 50568, cost: 2076017.0
Run 1, iteration: 2/100, moves: 5711, cost: 2074930.0
Run 1, iteration: 3/100, moves: 18, cost: 2074929.0
Run 1, iteration: 4/100, moves: 0, cost: 2074929.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 43550, cost: 2063030.0
Run 2, iteration: 2/100, moves: 396, cost: 2063030.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 78786, cost: 2068461.0
Run 3, iteration: 2/100, moves: 19331, cost: 2053319.0
Run 3, iteration: 3/100, moves: 3773, cost: 2053319.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 71540, cost: 2065945.0
Run 4, iteration: 2/100, moves: 3617, cost: 2065944.0
Run 4, iteration: 3/100, moves: 0, cost: 2065944.0
Init: initializing centroids
Init: i

In [85]:
labelsDf(2012, 25, clusters2012_25[0]).to_csv("year2012_25clusters.csv")

In [87]:
clusters2012_25[1].to_csv("year2012_25clusterCentroids.csv")

In [88]:
clusters2013_5 = makeClusters(2013, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 94155, cost: 2247169.0
Run 1, iteration: 2/100, moves: 14045, cost: 2247169.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 78058, cost: 2278392.0
Run 2, iteration: 2/100, moves: 17121, cost: 2278391.0
Run 2, iteration: 3/100, moves: 0, cost: 2278391.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 105337, cost: 2310122.0
Run 3, iteration: 2/100, moves: 8607, cost: 2310121.0
Run 3, iteration: 3/100, moves: 0, cost: 2310121.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 74404, cost: 2288155.0
Run 4, iteration: 2/100, moves: 1786, cost: 2288155.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 108674, cost: 2251641.0
Run

In [92]:
labelsDf(2013, 5, clusters2013_5[0]).to_csv("year2013_5clusters.csv")

In [93]:
clusters2013_5[1].to_csv("year2013_5clusterCentroids.csv")

In [89]:
clusters2013_10 = makeClusters(2013, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 46457, cost: 2085120.0
Run 1, iteration: 2/100, moves: 3903, cost: 2085120.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 66121, cost: 2032127.0
Run 2, iteration: 2/100, moves: 7328, cost: 2032127.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 108427, cost: 2048268.0
Run 3, iteration: 2/100, moves: 19483, cost: 2041688.0
Run 3, iteration: 3/100, moves: 4002, cost: 2041688.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 88089, cost: 2081931.0
Run 4, iteration: 2/100, moves: 13982, cost: 2081931.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 74120, cost: 2107507.0
Run 5, iteration: 2/100, moves: 0, cost: 2107507.0
B

In [95]:
labelsDf(2013, 10, clusters2013_10[0]).to_csv("year2013_10clusters.csv")

In [97]:
clusters2013_10[1].to_csv("year2013_10clusterCentroids.csv")

In [90]:
clusters2013_25 = makeClusters(2013, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 97145, cost: 1857137.0
Run 1, iteration: 2/100, moves: 2324, cost: 1857137.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 53024, cost: 1895188.0
Run 2, iteration: 2/100, moves: 21759, cost: 1895187.0
Run 2, iteration: 3/100, moves: 0, cost: 1895187.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 78155, cost: 1900820.0
Run 3, iteration: 2/100, moves: 5152, cost: 1900817.0
Run 3, iteration: 3/100, moves: 0, cost: 1900817.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 58977, cost: 1855906.0
Run 4, iteration: 2/100, moves: 5226, cost: 1855904.0
Run 4, iteration: 3/100, moves: 0, cost: 1855904.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, ite

In [99]:
labelsDf(2013, 25, clusters2013_25[0]).to_csv("year2013_25clusters.csv")

In [101]:
clusters2013_25[1].to_csv("year2013_25clusterCentroids.csv")

In [102]:
clusters2014_5 = makeClusters(2014, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 72851, cost: 1957091.0
Run 1, iteration: 2/100, moves: 3611, cost: 1957091.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 48274, cost: 2005036.0
Run 2, iteration: 2/100, moves: 24396, cost: 1987203.0
Run 2, iteration: 3/100, moves: 3634, cost: 1987203.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 66789, cost: 2027390.0
Run 3, iteration: 2/100, moves: 36488, cost: 1971329.0
Run 3, iteration: 3/100, moves: 31985, cost: 1969966.0
Run 3, iteration: 4/100, moves: 1489, cost: 1969966.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 80237, cost: 2035040.0
Run 4, iteration: 2/100, moves: 31369, cost: 2018624.0
Run 4, iteration: 3/100, moves: 8709, cost: 2018624.0
Init: initializing centr

In [106]:
labelsDf(2014, 5, clusters2014_5[0]).to_csv("year2014_5clusters.csv")

In [108]:
clusters2014_5[1].to_csv("year2014_5clusterCentroids.csv")

In [103]:
clusters2014_10 = makeClusters(2014, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 66370, cost: 1849305.0
Run 1, iteration: 2/100, moves: 1843, cost: 1849305.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 63464, cost: 1847663.0
Run 2, iteration: 2/100, moves: 2587, cost: 1847662.0
Run 2, iteration: 3/100, moves: 0, cost: 1847662.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 82423, cost: 1867359.0
Run 3, iteration: 2/100, moves: 16218, cost: 1867357.0
Run 3, iteration: 3/100, moves: 0, cost: 1867357.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 98874, cost: 1807917.0
Run 4, iteration: 2/100, moves: 8090, cost: 1807915.0
Run 4, iteration: 3/100, moves: 0, cost: 1807915.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, ite

In [110]:
labelsDf(2014, 10, clusters2014_10[0]).to_csv("year2014_10clusters.csv")

In [111]:
clusters2014_10[1].to_csv("year2014_10clusterCentroids.csv")

In [104]:
clusters2014_25 = makeClusters(2014, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 52119, cost: 1691853.0
Run 1, iteration: 2/100, moves: 18638, cost: 1678158.0
Run 1, iteration: 3/100, moves: 11099, cost: 1678158.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 68334, cost: 1688869.0
Run 2, iteration: 2/100, moves: 14326, cost: 1688691.0
Run 2, iteration: 3/100, moves: 1095, cost: 1688691.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 39651, cost: 1702930.0
Run 3, iteration: 2/100, moves: 1182, cost: 1702930.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 68784, cost: 1706721.0
Run 4, iteration: 2/100, moves: 6727, cost: 1700620.0
Run 4, iteration: 3/100, moves: 303, cost: 1700620.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...


In [114]:
labelsDf(2014, 25, clusters2014_25[0]).to_csv("year2014_25clusters.csv")

In [115]:
clusters2014_25[1].to_csv("year2014_25clusterCentroids.csv")

In [116]:
clusters2015_5 = makeClusters(2015, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 64130, cost: 1954481.0
Run 1, iteration: 2/100, moves: 3479, cost: 1954481.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 48285, cost: 1945410.0
Run 2, iteration: 2/100, moves: 19715, cost: 1944088.0
Run 2, iteration: 3/100, moves: 1457, cost: 1944088.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 63197, cost: 1891818.0
Run 3, iteration: 2/100, moves: 23079, cost: 1889970.0
Run 3, iteration: 3/100, moves: 1971, cost: 1889970.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 33262, cost: 1895711.0
Run 4, iteration: 2/100, moves: 8777, cost: 1894543.0
Run 4, iteration: 3/100, moves: 1298, cost: 1894543.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...


In [120]:
labelsDf(2015, 5, clusters2015_5[0]).to_csv("year2015_5clusters.csv")

In [121]:
clusters2015_5[1].to_csv("year2015_5clusterCentroids.csv")

In [117]:
clusters2015_10 = makeClusters(2015, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 54670, cost: 1829253.0
Run 1, iteration: 2/100, moves: 764, cost: 1829253.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 81055, cost: 1763264.0
Run 2, iteration: 2/100, moves: 5300, cost: 1763264.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 81254, cost: 1749706.0
Run 3, iteration: 2/100, moves: 1722, cost: 1749706.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 34825, cost: 1773536.0
Run 4, iteration: 2/100, moves: 4325, cost: 1773534.0
Run 4, iteration: 3/100, moves: 0, cost: 1773534.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 45533, cost: 1759429.0
Run 5, iteration: 2/100, moves: 25897, cost: 1759429.0
Best

In [123]:
labelsDf(2015, 10, clusters2015_10[0]).to_csv("year2015_10clusters.csv")

In [126]:
clusters2015_10[1].to_csv("year2015_10clusterCentroids.csv")

In [118]:
clusters2015_25 = makeClusters(2015, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 36605, cost: 1605746.0
Run 1, iteration: 2/100, moves: 3064, cost: 1605746.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 60883, cost: 1609663.0
Run 2, iteration: 2/100, moves: 6458, cost: 1606365.0
Run 2, iteration: 3/100, moves: 2330, cost: 1606365.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 45390, cost: 1646569.0
Run 3, iteration: 2/100, moves: 11435, cost: 1646568.0
Run 3, iteration: 3/100, moves: 0, cost: 1646568.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 67411, cost: 1618200.0
Run 4, iteration: 2/100, moves: 15942, cost: 1618197.0
Run 4, iteration: 3/100, moves: 0, cost: 1618197.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5,

In [128]:
labelsDf(2015, 25, clusters2015_25[0]).to_csv("year2015_25clusters.csv")

In [130]:
clusters2015_25[1].to_csv("year2015_25clusterCentroids.csv")

In [131]:
clusters2016_5 = makeClusters(2016, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 121212, cost: 1958239.0
Run 1, iteration: 2/100, moves: 18431, cost: 1938579.0
Run 1, iteration: 3/100, moves: 5032, cost: 1938579.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 71801, cost: 1932847.0
Run 2, iteration: 2/100, moves: 18656, cost: 1921344.0
Run 2, iteration: 3/100, moves: 9302, cost: 1921344.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 110708, cost: 1933009.0
Run 3, iteration: 2/100, moves: 633, cost: 1933009.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 33990, cost: 1960315.0
Run 4, iteration: 2/100, moves: 3655, cost: 1960315.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 46086, cost: 1954258.

In [135]:
labelsDf(2016, 5, clusters2016_5[0]).to_csv("year2016_5clusters.csv")

In [137]:
clusters2016_5[1].to_csv("year2016_5clusterCentroids.csv")

In [132]:
clusters2016_10 = makeClusters(2016, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 48858, cost: 1784089.0
Run 1, iteration: 2/100, moves: 1892, cost: 1784089.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 109384, cost: 1797952.0
Run 2, iteration: 2/100, moves: 9221, cost: 1796543.0
Run 2, iteration: 3/100, moves: 2026, cost: 1796543.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 57252, cost: 1801610.0
Run 3, iteration: 2/100, moves: 9954, cost: 1801609.0
Run 3, iteration: 3/100, moves: 0, cost: 1801609.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 15079, cost: 1851245.0
Run 4, iteration: 2/100, moves: 0, cost: 1851245.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 30738, cost: 1779661.0
Run 5,

In [139]:
labelsDf(2016, 10, clusters2016_10[0]).to_csv("year2016_10clusters.csv")

In [141]:
clusters2016_10[1].to_csv("year2016_10clusterCentroids.csv")

In [133]:
clusters2016_25 = makeClusters(2016, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 42797, cost: 1648838.0
Run 1, iteration: 2/100, moves: 6375, cost: 1648838.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 75618, cost: 1623526.0
Run 2, iteration: 2/100, moves: 16056, cost: 1613204.0
Run 2, iteration: 3/100, moves: 7253, cost: 1604118.0
Run 2, iteration: 4/100, moves: 13452, cost: 1604117.0
Run 2, iteration: 5/100, moves: 0, cost: 1604117.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 68103, cost: 1640699.0
Run 3, iteration: 2/100, moves: 8673, cost: 1640518.0
Run 3, iteration: 3/100, moves: 129, cost: 1640518.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 53471, cost: 1631263.0
Run 4, iteration: 2/100, moves: 5773, cost: 1631262.0
Run 4, iteration: 3/100, moves

In [143]:
labelsDf(2016, 25, clusters2016_25[0]).to_csv("year2016_25clusters.csv")

In [145]:
clusters2016_25[1].to_csv("year2016_25clusterCentroids.csv")

In [162]:
finalDf.sort('District', ascending = True)['District'].unique()

//anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  31.])

In [151]:
def makeDistrictClusters(district, num):
    df = filterDistrict(district)
    df = df.reset_index()
    if(num == 5):
        fitClusters = km_5.fit_predict(df)
        clusterCentroidsDf = pd.DataFrame(km_5.cluster_centroids_)
    elif(num == 10):
        fitClusters = km_10.fit_predict(df)
        clusterCentroidsDf = pd.DataFrame(km_10.cluster_centroids_)
    else:
        fitClusters = km_25.fit_predict(df)
        clusterCentroidsDf = pd.DataFrame(km_25.cluster_centroids_)
    clusterCentroidsDf.columns = df.columns
    return fitClusters, clusterCentroidsDf

In [ ]:
def districtLabelsDf(district, num, clustersDf):
    df = filterDistrict(district)
    df = df.reset_index()
    clustersDf = pd.DataFrame(clustersDf)
    if(num == 5):
        clustersDf.columns = ['clusters_5']
    elif(num == 10):
        clustersDf.columns = ['clusters_10']
    else:
        clustersDf.columns = ['clusters_25']
    combinedDf = pd.concat([df, clustersDf], axis = 1)
    return combinedDf

In [168]:
clustersDistrict1_5 = makeDistrictClusters(1, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 31819, cost: 2088796.0
Run 1, iteration: 2/100, moves: 5426, cost: 2088796.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 60827, cost: 2037308.0
Run 2, iteration: 2/100, moves: 7636, cost: 2037308.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 58255, cost: 2095224.0
Run 3, iteration: 2/100, moves: 1937, cost: 2095224.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 53569, cost: 2046210.0
Run 4, iteration: 2/100, moves: 6394, cost: 2044487.0
Run 4, iteration: 3/100, moves: 769, cost: 2044487.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 76754, cost: 2032668.0
Run 5, iteration: 2/100, moves: 14767, cost: 2032667.0
R

In [173]:
districtLabelsDf(1, 5, clustersDistrict1_5[0]).to_csv("district1_5clusters.csv")

In [175]:
clustersDistrict1_5[1].to_csv("district1_5clusterCentroids.csv")

In [169]:
clustersDistrict1_10 = makeDistrictClusters(1, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 80376, cost: 1912845.0
Run 1, iteration: 2/100, moves: 17150, cost: 1912845.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 92008, cost: 1962486.0
Run 2, iteration: 2/100, moves: 20760, cost: 1955855.0
Run 2, iteration: 3/100, moves: 1242, cost: 1955855.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 62513, cost: 1960992.0
Run 3, iteration: 2/100, moves: 23222, cost: 1954365.0
Run 3, iteration: 3/100, moves: 738, cost: 1954365.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 42822, cost: 1947416.0
Run 4, iteration: 2/100, moves: 4817, cost: 1947416.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 36406, cost: 1948526.0

In [177]:
districtLabelsDf(1, 10, clustersDistrict1_10[0]).to_csv("district1_10clusters.csv")

In [179]:
clustersDistrict1_10[1].to_csv("district1_10clusterCentroids.csv")

In [170]:
clustersDistrict1_25 = makeDistrictClusters(1, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 37698, cost: 1783501.0
Run 1, iteration: 2/100, moves: 2752, cost: 1783500.0
Run 1, iteration: 3/100, moves: 0, cost: 1783500.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 40212, cost: 1776695.0
Run 2, iteration: 2/100, moves: 5135, cost: 1776695.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 58812, cost: 1746239.0
Run 3, iteration: 2/100, moves: 9425, cost: 1745910.0
Run 3, iteration: 3/100, moves: 1037, cost: 1745910.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 45117, cost: 1755816.0
Run 4, iteration: 2/100, moves: 0, cost: 1755816.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 46011, cost: 1734627.0
Run 5, 

In [181]:
districtLabelsDf(1, 25, clustersDistrict1_25[0]).to_csv("district1_25clusters.csv")

In [185]:
clustersDistrict1_25[1].to_csv("district1_25clusterCentroids.csv")

In [186]:
clustersDistrict2_5 = makeDistrictClusters(2, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 84905, cost: 2728444.0
Run 1, iteration: 2/100, moves: 8495, cost: 2728444.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 45551, cost: 2741362.0
Run 2, iteration: 2/100, moves: 297, cost: 2741361.0
Run 2, iteration: 3/100, moves: 0, cost: 2741361.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 101260, cost: 2753050.0
Run 3, iteration: 2/100, moves: 46881, cost: 2708369.0
Run 3, iteration: 3/100, moves: 18206, cost: 2708369.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 72593, cost: 2783433.0
Run 4, iteration: 2/100, moves: 7255, cost: 2783433.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 118940, cost: 2780781.0
R

In [190]:
districtLabelsDf(2, 5, clustersDistrict2_5[0]).to_csv("district2_5clusters.csv")

In [192]:
clustersDistrict2_5[1].to_csv("district2_5clusterCentroids.csv")

In [187]:
clustersDistrict2_10 = makeDistrictClusters(2, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 90174, cost: 2529667.0
Run 1, iteration: 2/100, moves: 56021, cost: 2526815.0
Run 1, iteration: 3/100, moves: 2391, cost: 2526815.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 63324, cost: 2609128.0
Run 2, iteration: 2/100, moves: 383, cost: 2609127.0
Run 2, iteration: 3/100, moves: 0, cost: 2609127.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 101202, cost: 2515872.0
Run 3, iteration: 2/100, moves: 21469, cost: 2515872.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 92750, cost: 2613746.0
Run 4, iteration: 2/100, moves: 10481, cost: 2613746.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 106880, cost: 2592137.0


In [194]:
districtLabelsDf(2, 10, clustersDistrict2_10[0]).to_csv("district2_10clusters.csv")

In [196]:
clustersDistrict2_10[1].to_csv("district2_10clusterCentroids.csv")

In [188]:
clustersDistrict2_25 = makeDistrictClusters(2, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 92456, cost: 2372267.0
Run 1, iteration: 2/100, moves: 2433, cost: 2372267.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 93802, cost: 2309785.0
Run 2, iteration: 2/100, moves: 10968, cost: 2309785.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 93917, cost: 2301115.0
Run 3, iteration: 2/100, moves: 38709, cost: 2291136.0
Run 3, iteration: 3/100, moves: 6659, cost: 2291136.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 58407, cost: 2366532.0
Run 4, iteration: 2/100, moves: 23109, cost: 2363203.0
Run 4, iteration: 3/100, moves: 4720, cost: 2363203.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 86683, cost: 2289289.

In [198]:
districtLabelsDf(2, 25, clustersDistrict2_25[0]).to_csv("district2_25clusters.csv")

In [200]:
clustersDistrict2_25[1].to_csv("district2_25clusterCentroids.csv")

In [201]:
clustersDistrict3_5 = makeDistrictClusters(3, 5)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 168848, cost: 2933489.0
Run 1, iteration: 2/100, moves: 49247, cost: 2884010.0
Run 1, iteration: 3/100, moves: 2315, cost: 2884010.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 48430, cost: 2948137.0
Run 2, iteration: 2/100, moves: 461, cost: 2948137.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 47154, cost: 3028172.0
Run 3, iteration: 2/100, moves: 0, cost: 3028172.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 126253, cost: 2868656.0
Run 4, iteration: 2/100, moves: 35946, cost: 2868655.0
Run 4, iteration: 3/100, moves: 0, cost: 2868655.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 60628, cost: 2978969.0
Run 

In [202]:
clustersDistrict3_10 = makeDistrictClusters(3, 10)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 60719, cost: 2708094.0
Run 1, iteration: 2/100, moves: 11788, cost: 2706029.0
Run 1, iteration: 3/100, moves: 1692, cost: 2706029.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 96360, cost: 2721420.0
Run 2, iteration: 2/100, moves: 35443, cost: 2721420.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 100674, cost: 2787823.0
Run 3, iteration: 2/100, moves: 8493, cost: 2779148.0
Run 3, iteration: 3/100, moves: 14859, cost: 2768215.0
Run 3, iteration: 4/100, moves: 3148, cost: 2768214.0
Run 3, iteration: 5/100, moves: 0, cost: 2768214.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 142016, cost: 2694184.0
Run 4, iteration: 2/100, moves: 17162, cost: 2694182.0
Run 4, iteration: 3/100, 

In [203]:
clustersDistrict3_25 = makeDistrictClusters(3, 25)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 91178, cost: 2444348.0
Run 1, iteration: 2/100, moves: 23561, cost: 2440159.0
Run 1, iteration: 3/100, moves: 3188, cost: 2440158.0
Run 1, iteration: 4/100, moves: 0, cost: 2440158.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 122314, cost: 2471796.0
Run 2, iteration: 2/100, moves: 3537, cost: 2471795.0
Run 2, iteration: 3/100, moves: 0, cost: 2471795.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 108838, cost: 2409769.0
Run 3, iteration: 2/100, moves: 29216, cost: 2408559.0
Run 3, iteration: 3/100, moves: 559, cost: 2408557.0
Run 3, iteration: 4/100, moves: 0, cost: 2408557.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 71852, cost: 2447931.0
Run 4, iteration: 2/100, moves: 10